In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
atlas = src.graph.graph_from_json('Outputs/atlas.json')

In [3]:
filename = 'cities.json'

graph_cities = src.graph.graph_from_json(filename)

In [4]:
# Graph containing DC chargers

with open('Data/AFDC/evse_stations.json', 'r') as file:
    evse = json.load(file)

In [5]:
stations_raw = []

for station in evse['fuel_stations']:

    checks = (
        station['state'] == 'CA',
        station['ev_dc_fast_num'] is not None,
        station['access_code'] == 'public',
        station['status_code'] == 'E',
        
    )

    if all(checks):

        stations_raw.append(station)

In [6]:
df_stations = pd.DataFrame.from_dict(stations_raw)

In [7]:
node_attributes = {
    'x': 'lambda n: n["longitude"]',
    'y': 'lambda n: n["latitude"]',
    'n_dcfc': 'lambda n: n["ev_dc_fast_num"]',
    'n_acl2': 'lambda n: n["ev_level2_evse_num"]',
    'network': 'lambda n: n["ev_network"]',
    'name': 'lambda n: n["station_name"]',
    'address': 'lambda n: n["street_address"]',
    'city': 'lambda n: n["city"]',
    'state': 'lambda n: n["state"]',
    'zip': 'lambda n: n["zip"]',
    'access_code': 'lambda n: n["access_code"]',
    'status_code': 'lambda n: n["status_code"]',
    'type': 'lambda n: "station"',
}

nlg = src.graph.nlg_from_dataframe(df_stations, node_attributes)

graph_station = src.graph.graph_from_nlg(nlg)

mapping = {n: f'station_{n}' for n in graph_station.nodes}

graph_station = nx.relabel_nodes(graph_station, mapping)

In [8]:
deep_reload(src)

graph_to_atlas, atlas_to_graph = src.graph.node_assignment(graph_station, atlas)

def is_corridor(rn):
    
    rn = str(rn)
    
    return ('I' in rn) or ('U' in rn) or (rn in ['S1', 'S101', 'S101 P', 'S99', 'S395'])

for source, node in graph_station._node.items():

    atlas_node = graph_to_atlas[source]

    road_numbers = np.array(
        [l['road_number'] for l in atlas._adj[atlas_node].values()]
    )
    
    if any([is_corridor(rn) for rn in road_numbers]):

        node['corridor'] = 1

    else:

        node['corridor'] = 0

In [9]:
graph = nx.union(graph_cities, graph_station)

In [10]:
weights = {
    "distance": 400e3,
    "time": 28800,
}

graph = src.adjacency.adjacency(atlas, graph, weights, compute_all = True)

 [------->] (1921/1921) 100.00%, 6.89 it/s, 278.94 s elapsed, 0.00 s remaining

In [11]:
src.graph.graph_to_json(graph, 'Outputs/graph.json')